In [1]:
from awrams.utils.processing.extract import extract
from awrams.utils.gis import ShapefileDB,extent_from_record
from awrams.utils.extents import get_default_extent
import pandas as pd
import numpy as np
import pickle
import os
from itertools import product
import matplotlib.pyplot as plt

In [2]:
### Set the customer folder and shapefile name
shp_name = 'NRM_clusters'

In [3]:
## Data Request dir folder path
DataRequest_dir = '/g/data/er4/rs2140/data/'
dir_out = '/g/data/er4/zk6340/code/Script_Hydro-projections'

# Which cluster
which_cluster = 'Wet Tropics'

# Participating GCMs, Bias correction approaches, Emission scenarios
gcms = ['CNRM-CERFACS-CNRM-CM5','CSIRO-BOM-ACCESS1-0','MIROC-MIROC5','NOAA-GFDL-GFDL-ESM2M']
bias_corr = ['CSIRO-CCAM-r3355-r240x120-ISIMIP2b-AWAP','r240x120-ISIMIP2b-AWAP', 'r240x120-MRNBC-AWAP', 'r240x120-QME-AWAP']
emission = ['rcp45','rcp85']

In [4]:
## Read shape file
shp = DataRequest_dir + 'shapefiles/NRM_clusters/' + shp_name + '.shp'
sdb = ShapefileDB(shp)
sdf = sdb.get_records_df()
sdf

,OBJECTID,Shape_Area,Shape_Leng,code,label
0,1,28.309947,43.872364,CS,Central Slopes
1,2,34.176299,56.117544,EC,East Coast
2,4,45.299310,45.869349,MB,Murray Basin
3,5,124.655757,119.284033,MN,Monsoonal North
4,6,407.116072,137.457553,R,Rangelands
5,7,23.431035,59.887218,SS,Southern Slopes
6,8,50.647491,87.137068,SSWF,Southern and South-Western Flatlands
7,9,19.503388,45.896712,WT,Wet Tropics


In [5]:
emap = pickle.load(open(DataRequest_dir + 'shapefiles/NRM_clusters/' + shp_name + '.pkl', 'rb'))

In [6]:
emap

{'Central Slopes': origin: -24.9,146.3, shape: (182, 125), cell_size: 0.05,
 'East Coast': origin: -21.25,146.55, shape: (279, 144), cell_size: 0.05,
 'Murray Basin': origin: -32.2,138.55, shape: (119, 229), cell_size: 0.05,
 'Monsoonal North': origin: -10.85,119.05, shape: (273, 593), cell_size: 0.05,
 'Rangelands': origin: -16.95,112.85, shape: (329, 723), cell_size: 0.05,
 'Southern Slopes': origin: -34.2,140.95, shape: (193, 203), cell_size: 0.05,
 'Southern and South-Western Flatlands': origin: -26.8,113.5, shape: (194, 518), cell_size: 0.05,
 'Wet Tropics': origin: -10.0,141.35, shape: (245, 177), cell_size: 0.05}

In [7]:
cluster = {}
for key in emap:
    if which_cluster in key:
        cluster[key] = emap[key]

In [8]:
period_hist = pd.date_range('1 jan 1976', '31 dec 2005', freq='D')
#period_projected = pd.date_range('1 jan 2080', '31 dec 2099', freq='D')
period_projected = pd.date_range('2080-01-01 12:00:00', '2099-12-31 00:00:00', freq='D')

In [9]:
#Function to get to the path 
def get_path(parameter,which_gcm,which_emission,which_bias_corr):
    return(os.path.join('/g/data/wj02/COMPLIANT/HMINPUT/output/AUS-5/BoM/%s/%s/r1i1p1/%s/latest/day/%s'%(which_gcm,which_emission,which_bias_corr,parameter)))

In [10]:
#Function to get to the file
def get_file(parameter,which_gcm,which_emission,which_bias_corr):
    base_filename = '%s_AUS-5_%s_%s_r1i1p1_%s_v1_day_20060101-20991231.nc'%(parameter,which_gcm,which_emission,which_bias_corr)
    return(base_filename)

In [11]:
#Function to calculate return period
def return_period(data, T):
    xbar=np.mean(data)
    stdev=np.std(data)
    ff=np.sqrt(6)/3.1416*(-0.5772-np.log(-np.log(1-1/T))) 
    qt=xbar+stdev*ff
    return(qt)

In [12]:
var_name = 'rain_day'
input_path_awap = '/g/data/er4/data/CLIMATE/rain_day/'
pattern = input_path_awap + '%s*.nc' % var_name
ds_awap = extract(input_path_awap, pattern, var_name, cluster, period_hist)
awap_mean = ds_awap.resample(rule='A-DEC').sum().mean()[0]
awap_max = ds_awap.max()[0]
yr_max_awap = ds_awap.resample(rule='A-DEC').max()
awap_return_period = return_period(yr_max_awap, 20)[0]

In [13]:
values = [(i, j, k) for i, j, k in product(gcms, bias_corr, emission)]

In [14]:
variable = 'pr'
input_path_gcm = '/g/data/er4/zk6340/code/Script_Hydro-projections/'
input_file = get_file(variable,values[0][0],values[0][2],values[0][1])
pattern_gcm = input_path_gcm + '%s*.nc' % variable
rr_gcm = extract(input_path_gcm, pattern_gcm, variable, cluster, period_projected)#kgM-2s-1
dd_gcm = rr_gcm*86400
gcm_mean = dd_gcm.resample(rule='A-DEC').sum().mean()[0]
gcm_max = dd_gcm.max()[0]
annual_mean_change = ((awap_mean-gcm_mean)/awap_mean)*100
annual_max_day_change = ((awap_max-gcm_max)/awap_max)*100
dd = {'GCM':values[0][0],'Bias_correction':values[0][2],'Emission':values[0][1],'Annual_mean_prcp_change':annual_mean_change,'Max_1_day_prcp_change':annual_max_day_change}

Exception: No files found in /g/data/er4/zk6340/code/Script_Hydro-projections/ matching /g/data/er4/zk6340/code/Script_Hydro-projections/pr*.nc

In [ ]:
dd_gcm

In [15]:
variable = 'pr'
all_details = []
for val in enumerate(values): 
    input_path_gcm = get_path(variable,val[1][0],val[1][1],val[1][2])
    input_file = get_path(variable,val[1][0],val[1][1],val[1][2])
    pattern_gcm = input_path_gcm + input_file
    dd_gcm = extract(input_path_gcm, pattern_gcm, variable, cluster, period_projected)
    gcm_mean = ds_gcm.resample(rule='A-DEC').sum().mean()[0]
    gcm_max = ds_gcm.max()[0]
    annual_mean_change = ((awap_mean-gcm_mean)/awap_mean)*100
    annual_max_day_change = ((awap_max-gcm_max)/awap_max)*100
    dd = {'GCM':val[0],'Bias_correction':val[1],'Emission':val[2],'Annual_mean_prcp_change':annual_mean_change,'Max_1_day_prcp_change':annual_max_day_change}
    all_details.append(dd)
all_data = pd.DataFrame(all_details)

Exception: No files found in /g/data/wj02/COMPLIANT/HMINPUT/output/AUS-5/BoM/CNRM-CERFACS-CNRM-CM5/CSIRO-CCAM-r3355-r240x120-ISIMIP2b-AWAP/r1i1p1/rcp45/latest/day/pr matching /g/data/wj02/COMPLIANT/HMINPUT/output/AUS-5/BoM/CNRM-CERFACS-CNRM-CM5/CSIRO-CCAM-r3355-r240x120-ISIMIP2b-AWAP/r1i1p1/rcp45/latest/day/pr/g/data/wj02/COMPLIANT/HMINPUT/output/AUS-5/BoM/CNRM-CERFACS-CNRM-CM5/CSIRO-CCAM-r3355-r240x120-ISIMIP2b-AWAP/r1i1p1/rcp45/latest/day/pr

In [ ]:
/g/data/wj02/COMPLIANT/HMINPUT/output/AUS-5/BoM/CNRM-CERFACS-CNRM-CM5/rcp45/r1i1p1/CSIRO-CCAM-r3355-r240x120-ISIMIP2b-AWAP/latest/day/pr

In [ ]:
all_data.to_csv(dir_out + '.csv')

In [ ]:
# fig = plt.figure()
# ax = fig.add_axes([0,0,1,1])
# title = "NRM Region: %s"%(which_cluster)
# ax.set_title(title) 
# months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
# ax.set_ylabel('Total Rainfall (mm)')
# xx = ax.bar(months, df_m['Wet Tropics'], color = 'skyblue')
# ax.text(x = months, y = df_m['Wet Tropics'],label = df_m['Wet Tropics'], va='center', ha='right',cex = 0.8, col = 'k')
# output_file = 'Rainfall_climatology_%s.jpeg'%(which_cluster)
# plt.savefig(os.path.join('/g/data/er4/zk6340/Hydro_projection',output_file))
# plt.show()

In [ ]:
dir_in = '/g/data/er4/zk6340/code/Script_Hydro-projections'

In [ ]:
ds = pd.read_csv(os.path.join(dir_in, 'modified.csv'))

In [ ]:
ds

In [ ]:
import seaborn as sns

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=1)
fig.set_size_inches((18.5, 10.5))
sns.boxplot(x="category", y="values", hue="emission", data=ds, palette="Set3")
plt.ylabel("% Change in 2080-2099 relative to 1976-2005", size=14)
plt.xlabel("",size=14)
plt.title("NRM Region: Wet Tropics", size=16)
output_file = 'box_plot.jpeg' 
plt.savefig(os.path.join(dir_out,output_file))

In [ ]:
tips = sns.load_dataset("tips")

In [ ]:
tips

In [ ]:
ds.boxplot(column=['Annual_mean_prcp_change', 'Max_1_day_prcp_change', 'Return_period_change'])